In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq

In [ ]:
def db(x): return 20*np.log10(np.abs(x))
def to_range(x, min_val, max_val):
    x *= (max_val-min_val)/(x.max()-x.min())
    x -= x.min()-min_val
    #(x-x.min()-min_val)/(x.max()-x.min())*max_val

## Define frequencies and sampling lengths
It seems as if the carrier needs to be at least three times the signal frequency. There will be a sideband at $f_{carrier} - 2 f_{signal}$, and there need to be some leeway for filtering.

In [ ]:
periods = 200 # Of the lowest frequency waveform
resolution = 115 # Elements per cycle of the highest frequency waveform

carrier_frequency = 200e3
signal_frequency = 41.287623461e3

signal_min = -1
signal_max = 1

all_freqs = [carrier_frequency, signal_frequency]
fs = resolution*np.max(all_freqs)
n = int(np.ceil(fs/np.min(all_freqs))*periods)
f = fftfreq(n,1/fs)
t=np.arange(n)/fs

per_to_show = np.min([2,periods])
n_to_show = int(np.ceil(fs/np.min(all_freqs))*per_to_show)
t_idx = np.arange(n//2-n_to_show//2,n//2+n_to_show//2)

input_signal = np.sin(2*np.pi*t*signal_frequency)
win = signal.windows.hann(n)

db_max = 0
db_min = -60
f_min = 30e3
f_max = 300e3
#f_max = fs/2
f_scale = 'linear'

f_idx = (f>f_min) & (f<f_max)

## Filter specification
Some kind of lowpass filtering is required for a pwm to work properly. 
The demands of the filter will increase when the pwm "carrier" is closer to the bandwidth of interest. 

In [ ]:
lowpass_frequency = 60e3
lowstop_frequency = 200e3
lowstop_frequency = carrier_frequency
#lpf = signal.iirfilter(2, lowpass_frequency/fs*2, btype='low', ftype='ellip', rp=3, rs=60)
lpf = signal.iirdesign(
    wp=lowpass_frequency/(fs/2), 
    ws=lowstop_frequency/(fs/2),
    gpass=3, gstop=60, ftype='cheby2')
w,h = signal.freqz(lpf[0],lpf[1], worN=f/fs*2*np.pi)
plt.semilogx(w*fs/np.pi/2, db(h))
plt.plot([lowpass_frequency,lowpass_frequency],plt.ylim())
plt.plot([lowstop_frequency,lowstop_frequency],plt.ylim())
plt.ylim((-100,2))
plt.show()

## Simple PWM 
The PWM signal is 1 when the signal is higher than the carrier, otherwise -1.

In [ ]:
def PWM(sig, f_carr):
    carrier = signal.waveforms.sawtooth(t*f_carr*2*np.pi, 0.5)   
    to_range(carrier, signal_min, signal_max)
    to_range(sig, 0.5*signal_min, 0.5*signal_max)
    return np.where( sig>carrier, signal_max, signal_min)

pwm = PWM(input_signal, carrier_frequency)
pwm_low = signal.lfilter(lpf[0],lpf[1], pwm)
pwm_fft = fft(pwm*win)/win.sum()
pwm_low_fft = fft(pwm_low*win)/win.sum()

plt.subplot(2,2,1)
plt.plot(t[t_idx], pwm[t_idx])
plt.subplot(2,2,2)
plt.plot(t[t_idx], pwm_low[t_idx])
plt.subplot(2,2,3)
plt.plot(f[f_idx], db(pwm_fft[f_idx]))
plt.xscale(f_scale)
plt.ylim(ymin=db_min, ymax=db_max)
plt.subplot(2,2,4)
plt.plot(f[f_idx], db(pwm_low_fft[f_idx]))
plt.xscale(f_scale)
plt.ylim(ymin=db_min, ymax=db_max)
plt.show()

## Delta PWM
The output again has state 1 or -1. Now the state is integrated to a reference signal. If the reference signal differs too much from the input signal, the output state changes.

Note: Only check if the reference "hits" the boundary from the "inside", i.e. do not change the state to negative if the reference still is outside the region. Otherswise it might happen that the reference drop below the threshold which causes a state change, but during the next sample the reference does not change sufficiently and is still below the threshold. In such cases it is important NOT to change the state.

In [ ]:
def delta_PWM(sig, delta, ref_out=None):
    threshold = 0.1
    pwm = np.empty(sig.shape)
    if ref_out is None:
        ref = np.empty(sig.shape)
    else:
        ref = ref_out
    pwm[0] = 1
    ref[0] = sig[0]
    for idx in range(1,len(sig)):
        ref[idx] = ref[idx-1] + pwm[idx-1]*delta
        if pwm[idx-1]*(ref[idx]-sig[idx]) > threshold: 
            pwm[idx] = -pwm[idx-1]
        else:
            pwm[idx] = pwm[idx-1]
    return pwm

def delta_PWM2(sig, delta):
    # Equvalent to above with zero threshold.
    # Gives a bit stream that changes very fast, almost every sample.
    pwm = np.empty(sig.shape)
    ref = np.empty(sig.shape)
    pwm[0] = 1
    ref[0] = sig[0]
    for idx in range(1,len(sig)):
        pwm[idx] = np.sign( sig[idx-1]-ref[idx-1] + (ref[idx-1] == sig[idx-1]) ) # Comparison to handle eqial cases
        ref[idx] = ref[idx-1] + pwm[idx-1]*delta
    return pwm, ref

d_ref = np.empty(input_signal.shape)
d_pwm = delta_PWM(input_signal, 0.01, d_ref)
d_pwm_low = signal.lfilter(lpf[0], lpf[1], d_pwm)
d_pwm_fft = fft(d_pwm*win)/win.sum()
d_pwm_low_fft = fft(d_pwm_low*win)/win.sum()

plt.subplot(3,2,1)
plt.plot(t[t_idx], d_pwm[t_idx])
plt.subplot(3,2,2)
plt.plot(t[t_idx], d_pwm_low[t_idx])
plt.subplot(3,2,3)
plt.plot(f[f_idx], db(d_pwm_fft[f_idx]))
plt.xscale(f_scale)
plt.ylim(ymin=db_min, ymax=db_max)
plt.subplot(3,2,4)
plt.plot(f[f_idx], db(d_pwm_low_fft[f_idx]))
plt.xscale(f_scale)
plt.ylim(ymin=db_min, ymax=db_max)
plt.subplot(3,1,3)
num_idx = 500
plt.plot(t[:num_idx],input_signal[:num_idx])
plt.plot(t[:num_idx],input_signal[:num_idx]+0.1,'green')
plt.plot(t[:num_idx],input_signal[:num_idx]-0.1,'green')
plt.plot(t[:num_idx], d_ref[:num_idx])
plt.show()

# Delta-Sigma PWM


In [ ]:
def delta_sigma_PWM(sig, delta, ref_out=None, err_out=None):
    threshold = 1
    pwm = np.empty(sig.shape)
    if ref_out is None:
        ref = np.empty(sig.shape)
    else:
        ref = ref_out
    if err_out is None:
        err = np.empty(sig.shape)
    else:
        err = err_out
    pwm[0] = 1
    err[0] = sig[0]-pwm[0]
    ref[0] = 0
    
    for idx in range(1,len(sig)):
        ref[idx] = ref[idx-1] - delta*err[idx-1]
        if pwm[idx-1]*ref[idx] > threshold: 
            pwm[idx] = -pwm[idx-1]
        else:
            pwm[idx] = pwm[idx-1]
        err[idx] = sig[idx] - pwm[idx]
    return pwm

ds_ref = np.empty(input_signal.shape)
ds_err = np.empty(input_signal.shape)
ds_pwm = delta_sigma_PWM(input_signal, 0.1, ds_ref, ds_err)

ds_pwm_low = signal.lfilter(lpf[0], lpf[1], ds_pwm)
ds_pwm_fft = fft(ds_pwm*win)/win.sum()
ds_pwm_low_fft = fft(ds_pwm_low*win)/win.sum()

plt.subplot(4,2,1)
plt.plot(t[t_idx], ds_pwm[t_idx])
plt.subplot(4,2,2)
plt.plot(t[t_idx], ds_pwm_low[t_idx])
plt.subplot(4,2,3)
plt.plot(f[f_idx], db(ds_pwm_fft[f_idx]))
plt.xscale(f_scale)
plt.ylim(ymin=db_min, ymax=db_max)
plt.subplot(4,2,4)
plt.plot(f[f_idx], db(ds_pwm_low_fft[f_idx]))
plt.xscale(f_scale)
plt.ylim(ymin=db_min, ymax=db_max)
plt.subplot(4,1,3)
num_idx = 500
plt.plot(t[:num_idx],input_signal[:num_idx])
#plt.plot(t[:num_idx],input_signal[:num_idx]+0.1,'green')
#plt.plot(t[:num_idx],input_signal[:num_idx]-0.1,'green')
plt.plot(t[:num_idx], ds_err[:num_idx])
plt.subplot(4,1,4)
plt.plot(ds_ref[:num_idx])
plt.plot(np.ones((num_idx,1)), 'green')
plt.plot(-np.ones((num_idx,1)), 'green')
plt.show()
        
